In [1]:
import io, time, json
import requests
from pathlib import Path
from bs4 import BeautifulSoup
import json as JSON

# Get 1,000 from Yelp! and some features

In [3]:
#Using API to get the names, url and some features of the restaurants. Storing and running the function in data_1
api_key ='iVEiMvbM9IdSVFlJ-icXHQrF1Yw7OMC2vAlZ_8yBwf_Fza4GQ7ey-RMN4OG7ADO0YvKJkVMDv1e3xzN_3g9mtfkDoXMSpWc3TdJBcJP3tJE-rMgzlG-pl4iTau1uXXYx'

def all_restaurants(api_key, query, x,y):
    headers = {'Authorization': 'Bearer %s'%api_key}
    params = {'location': query,'categories':'restaurants'}
    res = requests.get('https://api.yelp.com/v3/businesses/search', params = params, headers=headers).json()
    aux1= []
    for i in range(y,x,20):
        time.sleep(2)
        res2 = requests.get('https://api.yelp.com/v3/businesses/search', params = {'location': query, 'categories':'restaurants', 'offset':i}, headers=headers).json()
        aux1.append(res2)
    aux2=[]
    for i in aux1:
        for t in(i['businesses']):
            aux2.append(t)
    return aux2

data_1 = all_restaurants(api_key,'Pittsburgh',1000,0)

# Scraping using BeautifulSoup, building in a function to handle the specifics of Yelp! webpage

In [1]:
#The function is build to extract the difference reviews for a given set of restaurants. The first part of the code gets the number of 
#reviews pages for a restaurants, using this as an input we scrap the 20% of the total number of pages.
#We did this for two reason, to make it computationally feasible and because we are going to use only ratings between 2018-2019
import math
def extract_reviews(url):
    alv = url
    aux4 = requests.get(alv)
    url2 = BeautifulSoup(aux4.content, "html.parser")
    aux20=url2.find_all('div', attrs = {'class': "lemon--div__373c0__1mboc u-padding-b2 border-color--default__373c0__2oFDT text-align--center__373c0__1l506"})
    aux21 = aux20[0].find('span',attrs = {'class':'lemon--span__373c0__3997G text__373c0__2pB8f text-color--inherit__373c0__w_15m text-align--left__373c0__2pnx_'}).text
    aux22 = int(list(aux21.split(" "))[-1])
    aux22 = math.ceil(aux22*0.2) #20% of the total number of pages
    list3 = [i*20 for i in range(0,aux22)]
    list5 = [alv+"&start="+str(i) for i in list3]
    lst3 =[]
    for i in list5:
        aux11 = requests.get(i)
        aux6 = BeautifulSoup(aux11.content, "html.parser")
        aux7 = aux6.find_all('div',attrs={'itemprop':'review'})
        for i in aux7:
            aut = i.find('meta',attrs={'itemprop':'author'} ).get('content') #get author name
            date = i.find('meta',attrs={'itemprop':'datePublished'} ).get('content') #get date publish name
            des = i.find('p',attrs={'itemprop':'description'} ).text #get review
            rat = float(i.find('meta',attrs={'itemprop':'ratingValue'} ).get('content')) #get rating
            di = {'author':aut,'rating':rat,'date':date,'description':des}
            lst3.append(di) 
    return lst3

# Running the functions and saving in Json files

In [40]:
#This makes scraping easier and we can parallelize
reviews_dic_1 = {}
for i in data_1[0:50]:
    get_id = i['id']
    get_url = i['url']
    reviews = extract_reviews(url=get_url)
    reviews_dic_1[get_id] = reviews

reviews_dic_2 = {}
for i in data_1[50:250]:
    get_id = i['id']
    get_url = i['url']
    reviews = extract_reviews(url=get_url)
    reviews_dic_2[get_id] = reviews 

reviews_dic_3 = {}
for i in data_1[250:500]:
    get_id = i['id']
    get_url = i['url']
    reviews = extract_reviews(url=get_url)
    reviews_dic_3[get_id] = reviews   
    
reviews_dic_4 = {}
for i in data_1[500:750]:
    get_id = i['id']
    get_url = i['url']
    reviews = extract_reviews(url=get_url)
    reviews_dic_4[get_id] = reviews   
    
reviews_dic_5 = {}
for i in data_1[750:1000]:
    get_id = i['id']
    get_url = i['url']
    reviews = extract_reviews(url=get_url)
    reviews_dic_5[get_id] = reviews  
    
json1 = JSON.dumps(reviews_dic_1)
f = open("reviews_0_50.json","w")
f.write(json1)
f.close()

json2 = JSON.dumps(reviews_dic_2)
f = open("reviews_50_250.json","w")
f.write(json2)
f.close()

json3 = JSON.dumps(reviews_dic_3)
f = open("reviews_250_500.json","w")
f.write(json3)
f.close()

json4 = JSON.dumps(reviews_dic_4)
f = open("reviews_500_750.json","w")
f.write(json4)
f.close()

json5 = JSON.dumps(reviews_dic_5)
f = open("reviews_750_1000.json","w")
f.write(json5)
f.close()

json = JSON.dumps(data_1)
f = open("data_1.json","w")
f.write(json)
f.close()